# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f41c879d4c0>
├── 'a' --> tensor([[-1.7507, -0.9875,  0.0806],
│                   [ 0.2455,  1.1326, -0.2698]])
└── 'x' --> <FastTreeValue 0x7f41c879d3a0>
    └── 'c' --> tensor([[ 1.5896, -0.0469,  0.8529, -0.7100],
                        [-2.1702, -0.0126,  0.2901,  0.4673],
                        [ 0.4248, -0.4162,  0.0881, -0.3487]])

In [4]:
t.a

tensor([[-1.7507, -0.9875,  0.0806],
        [ 0.2455,  1.1326, -0.2698]])

In [5]:
%timeit t.a

61.7 ns ± 0.0595 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f41c879d4c0>
├── 'a' --> tensor([[ 0.2738, -0.2810,  0.5506],
│                   [ 1.2446, -2.0847, -0.6467]])
└── 'x' --> <FastTreeValue 0x7f41c879d3a0>
    └── 'c' --> tensor([[ 1.5896, -0.0469,  0.8529, -0.7100],
                        [-2.1702, -0.0126,  0.2901,  0.4673],
                        [ 0.4248, -0.4162,  0.0881, -0.3487]])

In [7]:
%timeit t.a = new_value

62.8 ns ± 0.408 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.7507, -0.9875,  0.0806],
               [ 0.2455,  1.1326, -0.2698]]),
    x: Batch(
           c: tensor([[ 1.5896, -0.0469,  0.8529, -0.7100],
                      [-2.1702, -0.0126,  0.2901,  0.4673],
                      [ 0.4248, -0.4162,  0.0881, -0.3487]]),
       ),
)

In [10]:
b.a

tensor([[-1.7507, -0.9875,  0.0806],
        [ 0.2455,  1.1326, -0.2698]])

In [11]:
%timeit b.a

56.5 ns ± 0.0612 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1817,  0.8708,  0.1509],
               [-0.7102, -2.3056, -0.7651]]),
    x: Batch(
           c: tensor([[ 1.5896, -0.0469,  0.8529, -0.7100],
                      [-2.1702, -0.0126,  0.2901,  0.4673],
                      [ 0.4248, -0.4162,  0.0881, -0.3487]]),
       ),
)

In [13]:
%timeit b.a = new_value

484 ns ± 0.314 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

857 ns ± 1.54 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 29.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 1.6 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

136 µs ± 240 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f411d4b9b80>
├── 'a' --> tensor([[[-1.7507, -0.9875,  0.0806],
│                    [ 0.2455,  1.1326, -0.2698]],
│           
│                   [[-1.7507, -0.9875,  0.0806],
│                    [ 0.2455,  1.1326, -0.2698]],
│           
│                   [[-1.7507, -0.9875,  0.0806],
│                    [ 0.2455,  1.1326, -0.2698]],
│           
│                   [[-1.7507, -0.9875,  0.0806],
│                    [ 0.2455,  1.1326, -0.2698]],
│           
│                   [[-1.7507, -0.9875,  0.0806],
│                    [ 0.2455,  1.1326, -0.2698]],
│           
│                   [[-1.7507, -0.9875,  0.0806],
│                    [ 0.2455,  1.1326, -0.2698]],
│           
│                   [[-1.7507, -0.9875,  0.0806],
│                    [ 0.2455,  1.1326, -0.2698]],
│           
│                   [[-1.7507, -0.9875,  0.0806],
│                    [ 0.2455,  1.1326, -0.2698]]])
└── 'x' --> <FastTreeValue 0x7f411d4b9df0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 90.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f411d534f70>
├── 'a' --> tensor([[-1.7507, -0.9875,  0.0806],
│                   [ 0.2455,  1.1326, -0.2698],
│                   [-1.7507, -0.9875,  0.0806],
│                   [ 0.2455,  1.1326, -0.2698],
│                   [-1.7507, -0.9875,  0.0806],
│                   [ 0.2455,  1.1326, -0.2698],
│                   [-1.7507, -0.9875,  0.0806],
│                   [ 0.2455,  1.1326, -0.2698],
│                   [-1.7507, -0.9875,  0.0806],
│                   [ 0.2455,  1.1326, -0.2698],
│                   [-1.7507, -0.9875,  0.0806],
│                   [ 0.2455,  1.1326, -0.2698],
│                   [-1.7507, -0.9875,  0.0806],
│                   [ 0.2455,  1.1326, -0.2698],
│                   [-1.7507, -0.9875,  0.0806],
│                   [ 0.2455,  1.1326, -0.2698]])
└── 'x' --> <FastTreeValue 0x7f41c87a3ca0>
    └── 'c' --> tensor([[ 1.5896, -0.0469,  0.8529, -0.7100],
                        [-2.1702, -0.0126,  0.2901,  0.4673],
                 

In [23]:
%timeit t_cat(trees)

29.8 µs ± 48.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.2 µs ± 69.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.5896, -0.0469,  0.8529, -0.7100],
                       [-2.1702, -0.0126,  0.2901,  0.4673],
                       [ 0.4248, -0.4162,  0.0881, -0.3487]],
              
                      [[ 1.5896, -0.0469,  0.8529, -0.7100],
                       [-2.1702, -0.0126,  0.2901,  0.4673],
                       [ 0.4248, -0.4162,  0.0881, -0.3487]],
              
                      [[ 1.5896, -0.0469,  0.8529, -0.7100],
                       [-2.1702, -0.0126,  0.2901,  0.4673],
                       [ 0.4248, -0.4162,  0.0881, -0.3487]],
              
                      [[ 1.5896, -0.0469,  0.8529, -0.7100],
                       [-2.1702, -0.0126,  0.2901,  0.4673],
                       [ 0.4248, -0.4162,  0.0881, -0.3487]],
              
                      [[ 1.5896, -0.0469,  0.8529, -0.7100],
                       [-2.1702, -0.0126,  0.2901,  0.4673],
                       [ 0.4248, -0.4162,  0.0881, -0.3487]],

In [26]:
%timeit Batch.stack(batches)

77.5 µs ± 211 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.5896, -0.0469,  0.8529, -0.7100],
                      [-2.1702, -0.0126,  0.2901,  0.4673],
                      [ 0.4248, -0.4162,  0.0881, -0.3487],
                      [ 1.5896, -0.0469,  0.8529, -0.7100],
                      [-2.1702, -0.0126,  0.2901,  0.4673],
                      [ 0.4248, -0.4162,  0.0881, -0.3487],
                      [ 1.5896, -0.0469,  0.8529, -0.7100],
                      [-2.1702, -0.0126,  0.2901,  0.4673],
                      [ 0.4248, -0.4162,  0.0881, -0.3487],
                      [ 1.5896, -0.0469,  0.8529, -0.7100],
                      [-2.1702, -0.0126,  0.2901,  0.4673],
                      [ 0.4248, -0.4162,  0.0881, -0.3487],
                      [ 1.5896, -0.0469,  0.8529, -0.7100],
                      [-2.1702, -0.0126,  0.2901,  0.4673],
                      [ 0.4248, -0.4162,  0.0881, -0.3487],
                      [ 1.5896, -0.0469,  0.8529, -0.7100],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 526 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

309 µs ± 4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
